In [1]:
import pandas as pd

In [2]:
load_file = "Resources/purchase_data.csv"

df=pd.read_csv(load_file)
df

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44
...,...,...,...,...,...,...,...
775,775,Aethedru70,21,Female,60,Wolf,3.54
776,776,Iral74,21,Male,164,Exiled Doomblade,1.63
777,777,Yathecal72,20,Male,67,"Celeste, Incarnation of the Corrupted",3.46
778,778,Sisur91,7,Male,92,Final Critic,4.19


In [3]:
player_count = len(df['SN'].unique())
pc = {'Total Players':[player_count]}
df1 = pd.DataFrame(pc)
df1

,Total Players
0,576


In [4]:
item_count = len(df['Item ID'].unique())
average_price = df['Price'].mean()
purchase_count = len(df['Purchase ID'].unique())
total_revenue = df['Price'].sum()

d = {'Number of Unique Items':[item_count],
       'Average Price':[average_price],
       'Number of Purchases':[purchase_count],
       'Total Revenue':[total_revenue]}
df2 = pd.DataFrame(d)
df2['Average Price']=df2['Average Price'].apply(lambda x: "${:.2f}".format(x))
df2['Total Revenue']=df2['Total Revenue'].apply(lambda x: "${:,.2f}".format(x))
df2

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,$3.05,780,"$2,379.77"


In [5]:
genders = df[['SN','Gender']]
genders2 = genders.drop_duplicates(subset="SN",keep='first')

male_count = (genders2.Gender.values == 'Male').sum()
female_count = (genders2.Gender.values == 'Female').sum()
other_count = (genders2.Gender.values == 'Other / Non-Disclosed').sum()
total_count = len(genders2['Gender'])

pct_male = male_count/total_count
pct_female = female_count/total_count
pct_other = other_count/total_count

g = [(male_count,pct_male),
    (female_count,pct_female),
    (other_count,pct_other)]

dfg = pd.DataFrame(g, columns=['Total Count', 'Percentage of Players'],
                   index = ['Male','Female','Other / Non-Disclosed'])
dfg['Percentage of Players']=dfg['Percentage of Players'].apply(lambda x: "{:.2f}%".format(x*100))
dfg

,Total Count,Percentage of Players
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%


In [6]:
#Run basic calculations to obtain purchase count,
#avg. purchase price, avg. purchase total per person etc. by gender
#Create a summary data frame to hold the results
gender_df2 = df.groupby('Gender')
gender_df3 = gender_df2.count()
gender_df4 = gender_df3[["Purchase ID"]]
gender_df5 = gender_df4.rename({'Purchase ID':'purchase_count'}, axis=1)
gender_df6 = gender_df2.sum()
gender_df7 = gender_df6[['Price']]
gender_df8 = gender_df7.rename({'Price':'total_purchase_value'}, axis=1)
gender_df9 = gender_df5.merge(gender_df8,on='Gender')
gender_df9['average_purchase_price'] = gender_df9.total_purchase_value / gender_df9.purchase_count
gender_df10 = genders2.groupby('Gender')
gender_df11 = gender_df10.count()
gender_df12 = gender_df9.merge(gender_df11,on='Gender')
gender_df12['avg_total_pur_per_person'] = gender_df12.total_purchase_value / gender_df12.SN
gender_df12 = gender_df12.drop(['SN'],axis=1)
gender_df12 = gender_df12[['purchase_count','average_purchase_price','total_purchase_value','avg_total_pur_per_person']]
gender_df12['average_purchase_price']=gender_df12['average_purchase_price'].apply(lambda x: f"${x:.2f}")
gender_df12['total_purchase_value']=gender_df12['total_purchase_value'].apply(lambda x: f"${x:,.2f}")
gender_df12['avg_total_pur_per_person']=gender_df12['avg_total_pur_per_person'].apply(lambda x: f"${x:.2f}")
gender_df12 = gender_df12.rename({'purchase_count':'Purchase Count',
                                 'average_purchase_price':'Average Purchase Price',
                                 'total_purchase_value':'Total Purchase Value',
                                 'avg_total_pur_per_person':'Avg Total Purchase per Person'}, axis=1)
gender_df12

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,"$1,967.64",$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


In [7]:
# Establish bins for ages
# Categorize the existing players using the age bins. Hint: use pd.cut()
# Calculate the numbers and percentages by age group
# Create a summary data frame to hold the results
# Optional: round the percentage column to two decimal points
# Display Age Demographics Table

age_df1 = df[['SN','Age']]
age_df2 = age_df1.drop_duplicates(subset="SN",keep='first')
bins = [0,9.9,14.9,19.9,24.9,29.9,34.9,39.9,120]
group_names = ["<10","10-14","15-19","20-24","25-29","30-34","35-39","40+"]
age_df2['Age Group'] = pd.cut(df["Age"], bins, labels=group_names, include_lowest=True)
age_df3 = age_df2.groupby('Age Group')
age_df3 = age_df3.count()
age_df3['Percentage of Players'] = age_df3.Age / total_count
age_df3 = age_df3.drop(['SN'],axis=1)
age_df3['Percentage of Players']=age_df3['Percentage of Players'].apply(lambda x: f"{x*100:.2f}%")
age_df3 = age_df3.rename({'Age':'Total Count'}, axis=1)
age_df3

C:\Users\srobi\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


,Total Count,Percentage of Players
Age Group,,
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


In [8]:
# Bin the purchase_data data frame by age
# Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below
# Create a summary data frame to hold the results
# Optional: give the displayed data cleaner formatting
# Display the summary data frame

pur_df1 = df[['Purchase ID','SN','Age','Price']]
pur_df2 = pur_df1.drop_duplicates(subset="SN",keep='first')
bins = [0,9.9,14.9,19.9,24.9,29.9,34.9,39.9,120]
group_names = ["<10","10-14","15-19","20-24","25-29","30-34","35-39","40+"]
pur_df1['Age Ranges'] = pd.cut(df["Age"], bins, labels=group_names, include_lowest=True)
pur_df2['Age Ranges'] = pd.cut(df["Age"], bins, labels=group_names, include_lowest=True)
pur_df3 = pur_df1.groupby('Age Ranges')
pur_df3=pur_df3.count()
pur_df3=pur_df3[['Price']]
pur_df4 = pur_df1.groupby('Age Ranges')
pur_df4=pur_df4.sum()
pur_df4=pur_df4[['Price']]
pur_df5 = pd.merge(pur_df3,pur_df4,on="Age Ranges",suffixes=("_3","_4"))
pur_df5['avg']=pur_df5['Price_4']/pur_df5['Price_3']
pur_df6 = pur_df2.groupby('Age Ranges')
pur_df6=pur_df6.count()
pur_df6=pur_df6[['Price']]
pur_df7 = pd.merge(pur_df5,pur_df6,on="Age Ranges",suffixes=("","_6"))
pur_df7['avg2']=pur_df7['Price_4']/pur_df7['Price']
pur_df7['Price_4']=pur_df7['Price_4'].apply(lambda x: f"${x:,.2f}")
pur_df7['avg']=pur_df7['avg'].apply(lambda x: f"${x:,.2f}")
pur_df7['avg2']=pur_df7['avg2'].apply(lambda x: f"${x:,.2f}")
pur_df7=pur_df7[['Price_3','avg','Price_4','avg2']]
pur_df7 = pur_df7.rename({'Price_3':'Purchase Count',
                                 'Price_4':'Total Purchase Value',
                                 'avg':'Average Purchase Price',
                                 'avg2':'Avg Total Purchase per Person'}, axis=1)
pur_df7

C:\Users\srobi\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
C:\Users\srobi\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Age Ranges,,,,
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,"$1,114.06",$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


In [9]:
# Top Spenders
# Run basic calculations to obtain the results in the table below
# Create a summary data frame to hold the results
# Sort the total purchase value column in descending order
# Optional: give the displayed data cleaner formatting
# Display a preview of the summary data frame
tsdf1 = df.groupby('SN')  # group by SN
tsdf1 = tsdf1.count()   # get count of purchases by SN
tsdf1 = tsdf1[['Purchase ID']]  # clean up the data frame
tsdf2 = df.groupby('SN')  # group by SN
tsdf2 = tsdf2.sum()  # get sum of purchases by SN
tsdf2 = tsdf2[['Price']]  # clean up the data frame
tsdf3 = pd.merge(tsdf1,tsdf2,on="SN") # merge price and count information
tsdf3['avg'] = tsdf3['Price'] / tsdf3['Purchase ID']  # add calculated column
tsdf3 = tsdf3[['Purchase ID','avg','Price']]  # reorder columns
tsdf3.sort_values('Price', ascending=False, inplace=True) # sort by total purchase value
tsdf3['avg']=tsdf3['avg'].apply(lambda x: f"${x:,.2f}") # clean up formatting
tsdf3['Price']=tsdf3['Price'].apply(lambda x: f"${x:,.2f}") # clean up formatting
tsdf3 = tsdf3.rename({'Purchase ID':'Purchase Count',
                                 'avg':'Average Purchase Price',
                                 'Price':'Total Purchase Value',
                                 }, axis=1)  # rename columns
tsdf3.head(5) # show the top 5 spenders

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


In [10]:
# Most Popular Items
# Retrieve the Item ID, Item Name, and Item Price columns
# Group by Item ID and Item Name. Perform calculations to obtain purchase count, average item price, and total purchase value
# Create a summary data frame to hold the results
# Sort the purchase count column in descending order
# Optional: give the displayed data cleaner formatting
# Display a preview of the summary data frame

itdf1 = df.groupby(['Item ID','Item Name'])  # group by item ID
itdf1 = itdf1.count()   # get count of purchases by item ID
itdf1 = itdf1[['Price']]  # clean up the data frame
itdf2 = df.groupby(['Item ID','Item Name'])  # group by item ID
itdf2 = itdf2.sum()   # get sum of purchases by item ID
itdf2 = itdf2[['Price']]  # clean up the data frame
itdf4 = df[['Item ID','Item Name','Price']] # need price and Item ID indexed together
itdf5 = itdf4.drop_duplicates(subset="Item ID",keep='first') # eliminate duplicates to get price and Item ID
itdf5 = itdf5.groupby(['Item ID','Item Name']) # groupby Item ID to merge dataframes
itdf5 = itdf5.sum()   # get list of Item IDs and their prices
itdf3 = pd.merge(itdf1,itdf5,on=["Item ID","Item Name"]) # merge the dataframes
itdf6 = pd.merge(itdf3,itdf2,on=["Item ID","Item Name"]) # merge the dataframes
itdf6.sort_values('Price_x', ascending=False, inplace=True) # sort by purchase count
itdf6['Price_y']=itdf6['Price_y'].apply(lambda x: f"${x:,.2f}") # clean up formatting
itdf6['Price']=itdf6['Price'].apply(lambda x: f"${x:,.2f}") # clean up formatting
itdf6 = itdf6.rename({'Price_x':'Purchase Count',
                                 'Price_y':'Item Price',
                                 'Price':'Total Purchase Value',
                                 }, axis=1)  # rename columns
itdf6.head(5) # show the top 5 most popular items

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.88,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
132,Persuasion,9,$3.19,$28.99
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77


In [11]:
itdf1 = df.groupby(['Item ID','Item Name'])  # group by item ID
itdf1 = itdf1.count()   # get count of purchases by item ID
itdf1 = itdf1[['Price']]  # clean up the data frame
itdf2 = df.groupby(['Item ID','Item Name'])  # group by item ID
itdf2 = itdf2.sum()   # get sum of purchases by item ID
itdf2 = itdf2[['Price']]  # clean up the data frame
itdf4 = df[['Item ID','Item Name','Price']] # need price and Item ID indexed together
itdf5 = itdf4.drop_duplicates(subset="Item ID",keep='first') # eliminate duplicates to get price and Item ID
itdf5 = itdf5.groupby(['Item ID','Item Name']) # groupby Item ID to merge dataframes
itdf5 = itdf5.sum()   # get list of Item IDs and their prices
itdf3 = pd.merge(itdf1,itdf5,on=["Item ID","Item Name"]) # merge the dataframes
itdf6 = pd.merge(itdf3,itdf2,on=["Item ID","Item Name"]) # merge the dataframes
itdf6.sort_values('Price', ascending=False, inplace=True) # sort by purchase value
itdf6['Price_y']=itdf6['Price_y'].apply(lambda x: f"${x:,.2f}") # clean up formatting
itdf6['Price']=itdf6['Price'].apply(lambda x: f"${x:,.2f}") # clean up formatting
itdf6 = itdf6.rename({'Price_x':'Purchase Count',
                                 'Price_y':'Item Price',
                                 'Price':'Total Purchase Value',
                                 }, axis=1)  # rename columns
itdf6.head(5) # show the top 5 most popular items

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.88,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
103,Singed Scalpel,8,$4.35,$34.80
